In [3]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50
import keras_tuner as kt 
import pandas as pd
import numpy as np

%run preprocess.ipynb

from training_history import enable_auto_export_csv

# export setiap run ke file 'training_history.csv' (set ke 1 untuk export langsung)
enable_auto_export_csv(1, 'training_history.csv')

Step 1: Splitting data...
Loading datasets...
Found 366 files belonging to 3 classes.
Found 61 files belonging to 3 classes.
Found 32 files belonging to 3 classes.

Classes found: ['1', '2', '3']
Class to PPB mapping: {0: 1.0, 1: 2.0, 2: 3.0}
Step 2: Converting labels to PPB values...
Step 3: Setting up data augmentation...
Step 4: Optimizing data pipeline...
Data preprocessing completed!


In [4]:
print("Step 6: Setting up training callbacks...")

# Training callbacks for better training control
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_mae',
        patience=15, #jumlah epoch yg ditunggu
        restore_best_weights=True, 
        verbose=1 #menampilkan pesan di konsol saat pelatihan dihentikan
    ), # Stop training if no improvement in validation loss
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5, #kurangi LR 50%
        patience=10,
        min_lr=1e-8,
        verbose=1
    ), # Reduce learning rate if no improvement
    tf.keras.callbacks.ModelCheckpoint(
        'best_aflatoxin_resnet50.keras',
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=False,
        verbose=1
    ) # Save best model
]

Step 6: Setting up training callbacks...


In [5]:
# Definisikan Global Variables yang akan digunakan di model.fit
INITIAL_EPOCHS = 25 
FINE_TUNE_EPOCHS = 30 
AUTOTUNE = tf.data.AUTOTUNE # Asumsi AUTOTUNE didefinisikan di preprocess.ipynb

In [6]:
# Fungsi Pembangunan Model untuk Keras Tuner (Menggantikan Kode Model Asli)

def build_model_for_tuning(hp):
    # --- 1. DEFENISI BASE MODEL ---
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    # 2. Setup FREEZE/UNFREEZE
    # TUNE: Fine-Tuning Depth (Lapisan Terdalam ResNet)
    fine_tune_at = hp.Choice('fine_tune_at', values=[170, 140]) 
    
    # Set semua trainable=True dulu, lalu bekukan yang awal
    base_model.trainable = True 
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False
        
    # --- 3. PEMBANGUNAN HEAD LAYER ---
    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    
    # TUNE: Jumlah Neuron di Lapisan Dense Pertama (L1) - Simplifikasi Model
    hp_units_l1 = hp.Choice('units_l1', values=[128, 64, 32])
    x = layers.Dense(units=hp_units_l1, activation='relu', name='hp_dense_1')(x)
    
    # TUNE: Dropout Rate
    hp_dropout = hp.Float('dropout_rate', min_value=0.2, max_value=0.4, step=0.1)
    x = layers.Dropout(rate=hp_dropout, name='hp_dropout')(x)
    
    # Tambahkan Dense Layer 2 (opsional, tetapi tetap kecil)
    x = layers.Dense(32, activation='relu', name='dense_32_fixed')(x) 
    
    # Lapisan Regresi Akhir (Tetap)
    outputs = layers.Dense(1, activation='linear', name='aflatoxin_output')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='HP_Estimator')

    # --- 4. KOMPILASI MODEL (Menggunakan LR Tuning) ---
    
    # TUNE: Learning Rate Phase 2
    hp_lr = hp.Choice('learning_rate', values=[1e-7, 5e-7, 1e-6, 5e-6]) 
    
    # TUNE: Optimizer
    hp_optimizer_name = hp.Choice('optimizer', values=['Adam', 'RMSprop'])
    
    if hp_optimizer_name == 'RMSprop':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=hp_lr)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=hp_lr)

    model.compile(
        optimizer=optimizer,
        loss='mse',
        metrics=['mae', 'mse']
    )
    
    return model

# Setup Model Checkpoint untuk Keras Tuner (Hanya simpan yang terbaik)
tuner_callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_mae', # Fokus pada MAE (lebih mudah diinterpretasikan)
        patience=10, 
        restore_best_weights=True,
        verbose=1
    ),
]

# Inisialisasi Keras Tuner (Random Search)
tuner = kt.RandomSearch(
    build_model_for_tuning,
    objective='val_mae', # Target: Meminimalkan Mean Absolute Error pada data validasi
    max_trials=30,       # Coba 30 kombinasi parameter yang berbeda
    executions_per_trial=1, # Tiap kombinasi dilatih 1 kali
    directory='aflatoxin_tuning_results',
    project_name='ResNet_Aflatoxin_Reg'
)

Reloading Tuner from aflatoxin_tuning_results\ResNet_Aflatoxin_Reg\tuner0.json


In [7]:
# Cell 3 & 4 (DIGABUNG dan DIUBAH total)

# 1. Pastikan model dengan LR 1e-6 sudah dilatih dan disimpan
# ASUMSI: File 'best_aflatoxin_resnet50.keras' berisi bobot terbaik Fase 1 (Frozen Base)

# 2. Muat Bobot Terbaik FASE 1
# Muat model terbaik Fase 1 (Frozen Base) untuk dijadikan bobot awal pada semua trial RS.
# Ini penting karena RS harus mulai dari titik terbaik yang sudah stabil.
best_frozen_model = tf.keras.models.load_model('best_aflatoxin_resnet50.keras')
initial_weights = best_frozen_model.get_weights()


print("Step 7: Starting Random Search for Fine-Tuning Hyperparameters...")

# --- JALANKAN SEARCH ---
# Kita menjalankan RS dengan epochs total Phase 2 (Fine-Tuning)
tuner.search(
    train_ds,
    validation_data=val_ds,
    epochs=FINE_TUNE_EPOCHS, 
    callbacks=tuner_callbacks,
    # Menggunakan Initial Weights dari Fase 1 untuk semua trial:
    # Keras Tuner akan menggunakan weights ini untuk inisialisasi pada build_model_for_tuning
    # Namun, karena tidak ada argumen `initial_weights` di tuner.search, 
    # kita harus memodifikasi fungsi build_model_for_tuning untuk memuatnya secara eksplisit
    # (Ini membutuhkan penyesuaian yang kompleks, jadi kita fokus pada tuning LR/Dropout/Unit)
    verbose=1
)

print("\nStep 8: Getting the best model...")
# Ambil model terbaik berdasarkan MAE Validasi
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.get_best_models(num_models=1)[0]

print("\n--- HASIL HYPERPARAMETER TERBAIK ---")
print(best_hp.values)

Trial 30 Complete [00h 06m 46s]
val_mae: 0.553886353969574

Best val_mae So Far: 0.553886353969574
Total elapsed time: 02h 59m 08s

Step 8: Getting the best model...

--- HASIL HYPERPARAMETER TERBAIK ---
{'fine_tune_at': 140, 'units_l1': 128, 'dropout_rate': 0.30000000000000004, 'learning_rate': 5e-06, 'optimizer': 'RMSprop'}


c:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 50 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [8]:
# Cell 9 (DIGANTI TOTAL, Menjadi Post-Tuning Analysis)

print("Step 9: Analyzing Random Search Results...")

# Ambil model terbaik berdasarkan MAE Validasi (setelah Random Search selesai)
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.get_best_models(num_models=1)[0]

print("\n--- HYPERPARAMETER TERBAIK DITEMUKAN ---")
print(f"LR: {best_hp.get('learning_rate'):.1e}")
print(f"Dropout Rate: {best_hp.get('dropout_rate'):.1f}")
print(f"Units L1: {best_hp.get('units_l1')}")
print(f"Fine Tune At: {best_hp.get('fine_tune_at')}")

# Jika Anda ingin melanjutkan pelatihan dengan HP terbaik ini (Fase 2)
# Anda harus memuat bobot Phase 1 terbaik dan melatih ulang dengan HP yang ditemukan.

# model = best_model
# print("Model sekarang menggunakan HP terbaik dari Random Search.")

Step 9: Analyzing Random Search Results...

--- HYPERPARAMETER TERBAIK DITEMUKAN ---
LR: 5.0e-06
Dropout Rate: 0.3
Units L1: 128
Fine Tune At: 140


In [ ]:
print("Step 9a: Generating Comparison Table for All Random Search Trials...")

# --- Ambil semua trial yang telah dijalankan (misalnya 30 trial) ---
# Gunakan max_trials dari tuner (30) atau angka yang cukup besar
all_trials = tuner.get_best_trials(num_trials=30) 
trial_data = []

# Ekstrak Hyperparameter dan Skor dari setiap trial
for trial in all_trials:
    trial_id = trial.trial_id
    hp = trial.hyperparameters.values
    
    # Ambil skor terbaik (MAE) dari trial tersebut
    # Keras Tuner menyimpan skor final dari EarlyStopping/best epoch
    score = trial.best_trial.score 
    
    # Kumpulkan data ke dalam dictionary
    trial_record = {
        'Trial ID': trial_id,
        'val_MAE': score,
        'LR (Phase 2)': hp.get('learning_rate'),
        'Optimizer': hp.get('optimizer'),
        'Units L1': hp.get('units_l1'),
        'Dropout': hp.get('dropout_rate'),
        'Fine Tune At': hp.get('fine_tune_at')
    }
    trial_data.append(trial_record)

# --- Konversi ke DataFrame dan Tampilkan ---
df_results = pd.DataFrame(trial_data)

# Urutkan berdasarkan val_MAE (Semakin kecil semakin baik)
df_results = df_results.sort_values(by='val_MAE', ascending=True)

# Format kolom numerik untuk tampilan yang lebih rapi
df_results['val_MAE'] = df_results['val_MAE'].round(4)
df_results['LR (Phase 2)'] = df_results['LR (Phase 2)'].apply(lambda x: f'{x:.1e}')
df_results['Dropout'] = df_results['Dropout'].round(2)


print("\n--- PERBANDINGAN HASIL SEMUA TRIAL RANDOM SEARCH (TERBAIK KE TERBURUK) ---")
# Reset index untuk tampilan ranking
df_results = df_results.reset_index(drop=True)
df_results.index.name = 'Rank'
df_results.index += 1 # Mulai dari Rank 1

# Tampilkan tabel
print(df_results.to_markdown())

Step 9a: Generating Comparison Table for All Random Search Trials...


NameError: name 'build_model_for_tuning' is not defined